In [ ]:
import pandas as pd
movies = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/movies.csv")

Nos definimos aqui os caracteres que serao explorados e substituimos a antiga coluna titulo pela a nova coluna com os titulos limpos

In [ ]:
import re
def clean_title(title):
    title = re.sub("[^a-zA-Z0-9 ]", "", title)
    return title

Criamos uma nova coluna para aplicar os titulos limpos ao inves da antiga coluna

In [ ]:
movies["clean_title"] = movies["title"].apply(clean_title)


Aqui é a engine que faz a comparaçao se utilizando da ideia do TF-IDF que em suma melhora a unicidade dos nossos resultados que a IDF (Inverse document frequency) que faz com que os termos mais raros tenham valores compativeis para identificar a similaridade.
A gente usa biblioteca sklearn é para isso, que é uma biblioteca python de machine learning

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))

tfidf = vectorizer.fit_transform(movies["clean_title"])

Criamos a função para realizar a comparação e nos retornar com os 5 mais similares
---



In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def search(title):
    title = clean_title(title)
    query_vec = vectorizer.transform([title])
    similarity = cosine_similarity(query_vec, tfidf).flatten()
    indices = np.argpartition(similarity, -5)[-5:]
    results = movies.iloc[indices].iloc[::-1]

    return results

Criamos agora a caixa em que iremos receber a a o titulo e tambem mostramos para o usuario os 5 filmes mais parecidos.

In [ ]:
import ipywidgets as widgets
from IPython.display import display

movie_input = widgets.Text(
    value='Toy Story',
    description='Movie Title:',
    disabled=False
)
movie_list = widgets.Output()

def on_type(data):
    with movie_list:
        movie_list.clear_output()
        title = data["new"]
        if len(title) > 5:
            display(search(title))

movie_input.observe(on_type, names='value')


display(movie_input, movie_list)

Text(value='Toy Story', description='Movie Title:')

Output()

Agora vou importar outra database dos rating para podemos recomendar filmes similares para perfils similares

In [ ]:
movie_id = 89745

#def find_similar_movies(movie_id):
movie = movies[movies["movieId"] == movie_id]
ratings = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/ratings.csv")
ratings.dtypes

userId         int64
movieId        int64
rating       float64
timestamp      int64
dtype: object

Aqui definimos os parametros para criar uma boa recomendação:
selecionando os usuarios similares, olhando as recomendações que esse usuario fez, alem disso defini tambem que só me fosse passado filme que pelo menos 10% dos usuarios similares recomendaram, para podermos fazer uma recomendaçao melhor.
Alem disso para nos termos uma visao do nicho expecifico buscamos comparar com o rating de todos os usuarios.

In [ ]:
similar_users = ratings[(ratings["movieId"] == movie_id) & (ratings["rating"] > 4)]["userId"].unique()
similar_user_recs = ratings[(ratings["userId"].isin(similar_users)) & (ratings["rating"] > 4)]["movieId"]
similar_user_recs = similar_user_recs.value_counts() / len(similar_users)
similar_user_recs = similar_user_recs[similar_user_recs > .10]
all_users = ratings[(ratings["movieId"].isin(similar_user_recs.index)) & (ratings["rating"] > 4)]
all_user_recs = all_users["movieId"].value_counts() / len(all_users["userId"].unique())
rec_percentages = pd.concat([similar_user_recs, all_user_recs], axis=1)
rec_percentages.columns = ["similar", "all"]
rec_percentages["score"] = rec_percentages["similar"] / rec_percentages["all"]
rec_percentages = rec_percentages.sort_values("score", ascending=False)
rec_percentages.head(10).merge(movies, left_index=True, right_on="movieId")
def find_similar_movies(movie_id):
    similar_users = ratings[(ratings["movieId"] == movie_id) & (ratings["rating"] > 4)]["userId"].unique()
    similar_user_recs = ratings[(ratings["userId"].isin(similar_users)) & (ratings["rating"] > 4)]["movieId"]
    similar_user_recs = similar_user_recs.value_counts() / len(similar_users)

    similar_user_recs = similar_user_recs[similar_user_recs > .10]
    all_users = ratings[(ratings["movieId"].isin(similar_user_recs.index)) & (ratings["rating"] > 4)]
    all_user_recs = all_users["movieId"].value_counts() / len(all_users["userId"].unique())
    rec_percentages = pd.concat([similar_user_recs, all_user_recs], axis=1)
    rec_percentages.columns = ["similar", "all"]

    rec_percentages["score"] = rec_percentages["similar"] / rec_percentages["all"]
    rec_percentages = rec_percentages.sort_values("score", ascending=False)
    return rec_percentages.head(10).merge(movies, left_index=True, right_on="movieId")[["score", "title", "genres"]]

In [ ]:
import ipywidgets as widgets
from IPython.display import display

movie_name_input = widgets.Text(
    value='Toy Story',
    description='Movie Title:',
    disabled=False
)
recommendation_list = widgets.Output()

def on_type(data):
    with recommendation_list:
        recommendation_list.clear_output()
        title = data["new"]
        if len(title) > 5:
            results = search(title)
            movie_id = results.iloc[0]["movieId"]
            display(find_similar_movies(movie_id))

movie_name_input.observe(on_type, names='value')

display(movie_name_input, recommendation_list)

Text(value='Toy Story', description='Movie Title:')

Output()

F. Maxwell Harper and Joseph A. Konstan. 2015. The MovieLens Datasets: History and Context. ACM Transactions on Interactive Intelligent Systems (TiiS) 5, 4: 19:1–19:19. <https://doi.org/10.1145/2827872>

G. A. Dalaorao, A. M. Sison and R. P. Medina, "Integrating Collocation as TF-IDF Enhancement to Improve Classification Accuracy," 2019 IEEE 13th International Conference on Telecommunication Systems, Services, and Applications (TSSA), Bali, Indonesia, 2019, pp. 282-285, doi: 10.1109/TSSA48701.2019.8985458.
